# Edamam API
- goal is to be able to pass some ingredients to the API & get back recipes
- their [swagger docs](https://developer.edamam.com/edamam-docs-recipe-api)

## what's in this notebook:
- hit "recipe search" endpoint
- build useful recipe object

In [1]:
import json
import requests

import pandas as pd

## hit "recipe search" endpoint

In [2]:
# fetch api key from secrets.json file
with open("secrets.json", "rb") as f:
    key_file = json.loads(f.read())

app_id = key_file["app_id"]
app_key = key_file["key"]

- [sample reponse](/sample_response.json) from edamam "get recipes" endpoint with "q=cherry tomatoes, sourdough bread" and "ingr=3-7"

## build useful recipe object

In [3]:
def get_recipes(ingr: list[str], num_ingr: tuple[int, int], dish: str, meal: str, app_id: str, app_key: str):
    recipes_endpoint = "https://api.edamam.com/api/recipes/v2?type=public"
    ingredients = f"q={'%2C%20'.join(ingr)}"  # TODO: handle empty `ingr` list
    min_max_ingr = f"ingr={min(num_ingr)}-{max(num_ingr)}"  # TODO: handle empty or single-element `num_ingr`
    dish_type=f"dishType={dish}"  # TODO: set default `dish`?
    meal_type = f"mealType={meal}"  # TODO: set default `meal`?
    full_url = f"{recipes_endpoint}&{ingredients}&{min_max_ingr}&{dish_type}&app_id={app_id}&app_key={app_key}"
    response = requests.get(full_url)
    data = response.json()
    print(response.status_code)
    
    if response.status_code == 200 and data["count"] > 0:
        items = []
        for item in data["hits"]:
            recipe_dict = {}
            recipe_dict["title"] = item["recipe"]["label"]
            recipe_dict["link"] = item["recipe"]["url"]
            recipe_dict["servings"] = item["recipe"]["yield"]
            recipe_dict["cuisine"] = item["recipe"]["cuisineType"]
            recipe_dict["diet_labels"] = item["recipe"]["dietLabels"]
            recipe_dict["healh_labels"] = item["recipe"]["healthLabels"]
            recipe_dict["cautions"] = item["recipe"]["cautions"]
            recipe_dict["ingredient_lines"] = item["recipe"]["ingredientLines"]
            
            ingreds = []
            for ingr in item["recipe"]["ingredients"]:
                ingredient_details = {}
                ingredient_details["text"] = ingr["text"]
                ingredient_details["quantity"] = ingr["quantity"]
                ingredient_details["measure"] = ingr["measure"]
                ingredient_details["weight"] = ingr["weight"]
                ingredient_details["food"] = ingr["food"]
                ingredient_details["food_category"] = ingr["foodCategory"]
                ingreds.append(ingredient_details)
                recipe_dict["ingredient_details"] = ingreds
            items.append(recipe_dict)
        
    return pd.DataFrame.from_dict(items)

    

From the Edamam docs:

| dishType options | mealType options |
| --- | --- |
| Biscuits and cookies, Bread, Cereal, Condiments and sauces, Desserts, Drinks,  Main course, Pancake, Preps, Preserve, Salad, Sandwiches, Soup, Starter, Sweets | Breakfast, Dinner, Lunch, Snack, Teatime |

In [4]:
test_ingr = ["cherry%20tomatoes", "sourdough%20bread"]
test_min_max = (3,7)
test_dish = "Main%20course"
test_meal = "Breakfast"

In [5]:
df = get_recipes(ingr=test_ingr, num_ingr=test_min_max, dish=test_dish, meal=test_meal, app_id=app_id, app_key=app_key)

200


In [6]:
df.columns

Index(['title', 'link', 'servings', 'cuisine', 'diet_labels', 'healh_labels',
       'cautions', 'ingredient_lines', 'ingredient_details'],
      dtype='object')

In [7]:
df["ingredient_details"].head()

0    [{'text': '* 8 to 12 cherry tomato, depending ...
1    [{'text': '2 Count thick cut slices sourdough ...
2    [{'text': '2 cherry tomatoes or 1 roma tomato,...
3    [{'text': '6 eggs', 'quantity': 6.0, 'measure'...
4    [{'text': '5 cherry tomatoes', 'quantity': 5.0...
Name: ingredient_details, dtype: object